In [3]:
import os
import json
import pandas as pd
import traceback

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [13]:
key = os.getenv("GOOGLE_API_KEY")

In [14]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")

In [16]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
    create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
    Make sure the questions are not repeated and check all the questions to be conforming the text as well.
    Ensure to mke {number} MCQs with 4 options each.
    ### Response_JSON ###
    {response_json}
"""


In [17]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
}

In [18]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [30]:
quiz_chain = LLMChain(
    llm = llm,
    prompt = quiz_generation_prompt,
    output_key = "quiz",
    verbose = True
)

In [22]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of teh question and give a complete analysis of the quiz. only use at max 50 words for complexity, 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student's ability.
Quiz_MCQs:
{quiz}     

Check from an expert English writer of the above quiz:
"""

In [28]:
quiz_evaluation_prompt = PromptTemplate(input_variables = ["subject","quiz"], template=TEMPLATE2)

In [31]:
review_chain = LLMChain(llm=llm, prompt = quiz_evaluation_prompt,output_key="review", verbose=True)

In [32]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain,review_chain], input_variables=["text","number","subject","tone","response_json"], output_variables = ["quiz", "review"], verbose =True)

In [33]:
file_path = "E:\Ashu_Main\mcqGenerator\data.txt"

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_26056\2570227045.py:1: SyntaxWarning: invalid escape sequence '\A'
  file_path = "E:\Ashu_Main\mcqGenerator\data.txt"


In [35]:
with open(file_path,'r', encoding='utf-8') as file:
    TEXT = file.read()

In [37]:
print(TEXT)

Since deep learning and machine learning tend to be used interchangeably, it’s worth noting the nuances between the two. Machine learning, deep learning, and neural networks are all sub-fields of artificial intelligence. However, neural networks is actually a sub-field of machine learning, and deep learning is a sub-field of neural networks.

The way in which deep learning and machine learning differ is in how each algorithm learns. "Deep" machine learning can use labeled datasets, also known as supervised learning, to inform its algorithm, but it doesn’t necessarily require a labeled dataset. The deep learning process can ingest unstructured data in its raw form (e.g., text or images), and it can automatically determine the set of features which distinguish different categories of data from one another. This eliminates some of the human intervention required and enables the use of large amounts of data. You can think of deep learning as "scalable machine learning" as Lex Fridman notes

In [38]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [39]:
NUMBER = 5
SUBJECT = "Machine learning"
TONE = "simple"

In [40]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\ASUS\AppData\Local\Temp\ipykernel_26056\861068774.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Since deep learning and machine learning tend to be used interchangeably, it’s worth noting the nuances between the two. Machine learning, deep learning, and neural networks are all sub-fields of artificial intelligence. However, neural networks is actually a sub-field of machine learning, and deep learning is a sub-field of neural networks.

The way in which deep learning and machine learning differ is in how each algorithm learns. "Deep" machine learning can use labeled datasets, also known as supervised learning, to inform its algorithm, but it doesn’t necessarily require a labeled dataset. The deep learning process can ingest unstructured data in its raw form (e.g., text or images), and it can automatically determine the set of features which distinguish different categories of data from one another. This eliminates some of the human intervention required and enables the use 

In [41]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Token: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 2857
Prompt Token: 1415
Completion Tokens: 1442
Total Cost: 0.0


In [42]:
response

{'text': 'Since deep learning and machine learning tend to be used interchangeably, it’s worth noting the nuances between the two. Machine learning, deep learning, and neural networks are all sub-fields of artificial intelligence. However, neural networks is actually a sub-field of machine learning, and deep learning is a sub-field of neural networks.\n\nThe way in which deep learning and machine learning differ is in how each algorithm learns. "Deep" machine learning can use labeled datasets, also known as supervised learning, to inform its algorithm, but it doesn’t necessarily require a labeled dataset. The deep learning process can ingest unstructured data in its raw form (e.g., text or images), and it can automatically determine the set of features which distinguish different categories of data from one another. This eliminates some of the human intervention required and enables the use of large amounts of data. You can think of deep learning as "scalable machine learning" as Lex F

In [43]:
quiz = response.get("quiz")

In [45]:
quiz_json_str = response.get("quiz", "").strip("```json\n").strip("```")

In [47]:
quiz_data = json.loads(quiz_json_str)

In [48]:
print(quiz_data)

{'1': {'mcq': 'Which of the following correctly represents the relationship between AI, Machine Learning, Deep Learning, and Neural Networks?', 'options': {'a': 'AI is a sub-field of Machine Learning, which is a sub-field of Deep Learning, which contains Neural Networks.', 'b': 'Machine Learning is a sub-field of AI, which contains Neural Networks, and Deep Learning is a sub-field of Neural Networks.', 'c': 'Neural Networks is a sub-field of AI, which contains Machine Learning, and Deep Learning is a sub-field of Machine Learning.', 'd': 'AI encompasses Machine Learning, Neural Networks is a sub-field of Machine Learning, and Deep Learning is a sub-field of Neural Networks.'}, 'correct': 'd'}, '2': {'mcq': 'What is a key difference in how deep learning and classical machine learning algorithms learn?', 'options': {'a': 'Deep learning requires labeled datasets, while classical machine learning does not.', 'b': 'Classical machine learning can ingest unstructured data, while deep learning

In [50]:
quiz_table_data = []
for key,value in quiz_data.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices": options, "Correct": correct})

In [51]:
quiz_table_data

[{'MCQ': 'Which of the following correctly represents the relationship between AI, Machine Learning, Deep Learning, and Neural Networks?',
  'Choices': 'a: AI is a sub-field of Machine Learning, which is a sub-field of Deep Learning, which contains Neural Networks. | b: Machine Learning is a sub-field of AI, which contains Neural Networks, and Deep Learning is a sub-field of Neural Networks. | c: Neural Networks is a sub-field of AI, which contains Machine Learning, and Deep Learning is a sub-field of Machine Learning. | d: AI encompasses Machine Learning, Neural Networks is a sub-field of Machine Learning, and Deep Learning is a sub-field of Neural Networks.',
  'Correct': 'd'},
 {'MCQ': 'What is a key difference in how deep learning and classical machine learning algorithms learn?',
  'Choices': 'a: Deep learning requires labeled datasets, while classical machine learning does not. | b: Classical machine learning can ingest unstructured data, while deep learning requires structured d

In [54]:
quiz = pd.DataFrame(quiz_table_data)

In [55]:
quiz.to_csv("machine_learning.csv",index=False)